## PySpark DataFrames: Análisis del Censo

### **Introducción y objetivos del práctico**

Este *notebook* de PySpark vamos a trabajar con información del Censo 2023 de Uruguay.

Enlace de los *Microdatos Censo 2023 Anonimizados*: https://www.gub.uy/instituto-nacional-estadistica/politicas-y-gestion/microdatos-censo-2023-anonimizados


Conjunto que vamos a trabajar en esta clase: https://www5.ine.gub.uy/documents/CENSO%202023/Microdatos/personas_ext_26_02.rar


Cuestionario del censo: https://www.gub.uy/instituto-nacional-estadistica/sites/instituto-nacional-estadistica/files/2025-02/Cuestionario_censo2023%20%281%29.pdf


Diccionario de variables: https://www5.ine.gub.uy/documents/CENSO%202023/Microdatos/Diccionario%20de%20variables%202023.xlsx



Resumen del Proceso

- **Preparar el entorno:** Importar las bibliotecas necesarias y crear la sesión Spark.
- **Leer y preparar los datos:** Leer el archivo CSV, renombrar las columnas y transformar las características en un solo vector.
- **Dividir los datos:** Separar los datos en conjuntos de entrenamiento y prueba.
- **Entrenar el modelo:** Crear y entrenar el modelo de regresión lineal.
- **Evaluar el modelo:** Realizar predicciones y evaluar el rendimiento del modelo.

Con estos pasos, podrás construir y evaluar un modelo de regresión lineal en Spark.

In [ ]:
# !pip install pyspark pandas

### **Configuración del entorno (SparkSession)**


In [1]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import avg, max, min, round, count, col
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import expr
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, RobustScaler
from pyspark.sql.functions import desc
from pyspark.sql.functions import corr
from pyspark.ml.evaluation import RegressionEvaluator


### **Carga y exploración inicial de datos**


In [6]:
spark = SparkSession.builder \
    .config("spark.driver.memory", "2g") \
    .appName("MyApp") \
    .getOrCreate()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.
Sigan ejecutando las siguientes celdas de código.

In [3]:
# !apt-get update -y
# !apt-get install -y openjdk-17-jdk

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,289 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://cli.github.com/packages stable/main amd64 Packages [344 B]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,520 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease

In [4]:
# !pip install -q "pyspark>=3.6,<3.7"

ERROR: Could not find a version that satisfies the requirement pyspark<3.7,>=3.6 (from versions: 2.1.2, 2.1.3, 2.2.0.post0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.4.5, 2.4.6, 2.4.7, 2.4.8, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.1.1, 3.1.2, 3.1.3, 3.2.0, 3.2.1, 3.2.2, 3.2.3, 3.2.4, 3.3.0, 3.3.1, 3.3.2, 3.3.3, 3.3.4, 3.4.0, 3.4.1, 3.4.2, 3.4.3, 3.4.4, 3.5.0, 3.5.1, 3.5.2, 3.5.3, 3.5.4, 3.5.5, 3.5.6, 3.5.7, 4.0.0.dev1, 4.0.0.dev2, 4.0.0, 4.0.1, 4.1.0.dev1, 4.1.0.dev2, 4.1.0.dev3)
ERROR: No matching distribution found for pyspark<3.7,>=3.6


In [5]:
#import os

#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
#os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

#os.environ.pop("PYSPARK_SUBMIT_ARGS", None)
#os.environ.pop("SPARK_HOME", None)

In [7]:
sdf = spark.read.csv('personas_ext_26_02.csv', header=True, inferSchema=True)

In [8]:
print("Esquema del DataFrame de Spark:")
sdf.printSchema()

Esquema del DataFrame de Spark:
root
 |-- _c0: integer (nullable = true)
 |-- ID_CENSO: double (nullable = true)
 |-- DIRECCION_ID: string (nullable = true)
 |-- DEPARTAMENTO: integer (nullable = true)
 |-- LOCALIDAD: integer (nullable = true)
 |-- VIVID: string (nullable = true)
 |-- HOGID: string (nullable = true)
 |-- PERID: integer (nullable = true)
 |-- REGION_4: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- MUNICIPIO_PAIS: string (nullable = true)
 |-- TIPO_MUNICIPIO_PAIS: string (nullable = true)
 |-- FUENTE_EXT: integer (nullable = true)
 |-- SIT_CALLE: integer (nullable = true)
 |-- CUESTIONARIO_COMPLETO: integer (nullable = true)
 |-- CUESTIONARIO_BASICO: integer (nullable = true)
 |-- RRAA: integer (nullable = true)
 |-- UNIVERSO: integer (nullable = true)
 |-- VIVVO00: integer (nullable = true)
 |-- PERPH02: integer (nullable = true)
 |-- PERNA01: integer (nullable = true)
 |-- PERNA01_TRAMO: string (nullable = true)
 |-- PERPA01: integer (nullable = t

In [9]:
sdf.show(5)

+---+--------+------------+------------+---------+-----+-----+-----+--------+----+--------------+-------------------+----------+---------+---------------------+-------------------+----+--------+-------+-------+-------+-------------+-------+-------+---------+-------+---------+---------+---------+---------+---------+---------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+---------+---------+-------+-------+---------+-------+---------+---------+-------+---------+---------+-------+-------+-------+-------+-------+---------+---------+-------+---------+-------+-------+---------+-----------+---------+-----------+-------+-------+-------+-------+-------+-------+-------+-------+---------+---------+-------+---------+---------+-------+---------+---------+-------+---------+-------+--------+--------+------------------------+-----------+----------+---------------+----------+---------------+------------+-----------------+-------+------------+
|_c

### **Selección y filtrado de datos**

In [10]:
sdf.select('MUNICIPIO_PAIS').show(20)

+------------------+
|    MUNICIPIO_PAIS|
+------------------+
|       Municipio E|
|       Municipio E|
|       Municipio D|
|       Municipio D|
|       Municipio D|
|     Sin Municipio|
|       Las Piedras|
|       Municipio A|
|       Municipio A|
|     Sin Municipio|
|          Progreso|
|Ciudad de la Costa|
|        Piriápolis|
|     Sin Municipio|
|       Municipio C|
|     Sin Municipio|
|       Municipio E|
|       Municipio E|
|       Municipio E|
|       Municipio F|
+------------------+
only showing top 20 rows



In [11]:
sdf.select('MUNICIPIO_PAIS').distinct().show()

+--------------------+
|      MUNICIPIO_PAIS|
+--------------------+
|            Progreso|
|              Garzón|
|           Rodríguez|
|               Sauce|
|      Joaquín Suarez|
|        San Bautista|
|              Casupá|
|               Pando|
|         Municipio B|
|         Municipio F|
|Nicolich Ciudad L...|
|    Ciudad del Plata|
|   Ombúes de Lavalle|
|         Bella Unión|
|Quebracho - Paysandu|
|          Santa Rosa|
|           Castillos|
|           Mariscala|
|          San Carlos|
|   Florencio Sánchez|
+--------------------+
only showing top 20 rows



In [12]:
sdf.filter(sdf['MUNICIPIO_PAIS'] == "Municipio E").show(20)


+------+--------+------------+------------+---------+-----+-----+-----+--------+----+--------------+-------------------+----------+---------+---------------------+-------------------+----+--------+-------+-------+-------+-------------+-------+-------+---------+-------+---------+---------+---------+---------+---------+---------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+---------+---------+-------+-------+---------+-------+---------+---------+-------+---------+---------+-------+-------+-------+-------+-------+---------+---------+-------+---------+-------+-------+---------+-----------+---------+-----------+-------+-------+-------+-------+-------+-------+-------+-------+---------+---------+-------+---------+---------+-------+---------+---------+-------+---------+---------+--------+--------+------------------------+-----------+----------+---------------+----------+---------------+------------+-----------------+-------+------------

In [13]:
sdf.filter(sdf['DEPARTAMENTO'] < 4).show(20)

+---+--------+------------+------------+---------+-----+-----+-----+--------+----+------------------+-------------------+----------+---------+---------------------+-------------------+----+--------+-------+-------+-------+-------------+-------+-------+---------+-------+---------+---------+---------+---------+---------+---------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+---------+---------+-------+-------+---------+-------+---------+---------+-------+---------+---------+-------+-------+-------+-------+-------+---------+---------+-------+---------+-------+-------+---------+-----------+---------+-----------+-------+-------+-------+-------+-------+-------+-------+-------+---------+---------+-------+---------+---------+-------+---------+---------+-------+---------+-------+--------+--------+------------------------+-----------+----------+---------------+----------+---------------+------------+-----------------+-------+------------+

### **Transformación de columnas**

In [14]:
sdf_renombrado = sdf.select(
    col("ID_CENSO"),
    col("DEPARTAMENTO"),
    col("PERNA01").alias("Edad"),
    col("PERFM01").alias("Sexo"),
    col("MUNICIPIO_PAIS").alias("Municipio")
)

In [15]:
print("Esquema con columnas renombradas:")
sdf_renombrado.printSchema()


Esquema con columnas renombradas:
root
 |-- ID_CENSO: double (nullable = true)
 |-- DEPARTAMENTO: integer (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: integer (nullable = true)
 |-- Municipio: string (nullable = true)



In [16]:
print("Primeras filas con columnas renombradas:")
sdf_renombrado.show(5)


Primeras filas con columnas renombradas:
+--------+------------+----+----+-----------+
|ID_CENSO|DEPARTAMENTO|Edad|Sexo|  Municipio|
+--------+------------+----+----+-----------+
|     1.0|           1|  74|8888|Municipio E|
|     2.0|           1|  37|7777|Municipio E|
|     3.0|           1|  39|8888|Municipio D|
|     4.0|           1|  14|8888|Municipio D|
|     5.0|           1|  19|8888|Municipio D|
+--------+------------+----+----+-----------+
only showing top 5 rows



In [17]:
sdf_con_grupo = sdf_renombrado.withColumn(
    "Grupo_Edad",
    when(col("Edad") < 18, "Menor")
    .when((col("Edad") >= 18) & (col("Edad") < 65), "Adulto")
    .when(col("Edad") >= 65, "Adulto Mayor")
    .otherwise("Sin Dato")
)

In [18]:
print("DataFrame con la nueva columna 'Grupo_Edad':")
sdf_con_grupo.select("Edad", "Grupo_Edad").show(10)



DataFrame con la nueva columna 'Grupo_Edad':
+----+------------+
|Edad|  Grupo_Edad|
+----+------------+
|  74|Adulto Mayor|
|  37|      Adulto|
|  39|      Adulto|
|  14|       Menor|
|  19|      Adulto|
|5555|Adulto Mayor|
|  61|      Adulto|
|  30|      Adulto|
|  52|      Adulto|
|  52|      Adulto|
+----+------------+
only showing top 10 rows



In [19]:
sdf_con_grupo.select("Edad").describe().show()


+-------+------------------+
|summary|              Edad|
+-------+------------------+
|  count|           2569143|
|   mean| 972.6615926011125|
| stddev|2067.8504668778173|
|    min|                 0|
|    max|              5555|
+-------+------------------+



outliers= valore atìpico = valor extraño!


Limpieza y Filtrado de Valores Atípicos/Especiales

In [ ]:
sdf_limpio = sdf_renombrado.filter(
    (col("Edad") < 120) & (col("Edad") > 0)
)

total_original = sdf.count()
total_limpio = sdf_limpio.count()

print(f"Total de registros original: {total_original}")
print(f"Total de registros después de filtrar edades atípicas: {total_limpio}")
print(f"Registros eliminados: {total_original - total_limpio}")



### **Análisis descriptivo y agregaciones**

Cálculo de la edad promedio, máxima y mínima por municipio

In [ ]:
stats_por_municipio = sdf_renombrado.groupBy("Municipio").agg(
    round(avg("Edad"), 2).alias("Edad_Promedio"),
    max("Edad").alias("Edad_Maxima"),
    min("Edad").alias("Edad_Minima"),
    count("*").alias("Total_Personas")
)

stats_por_municipio.orderBy(col("Edad_Promedio").desc()).show(10)

PySpark SQL: Unificación de SQL con Spark

In [ ]:
sdf_renombrado.createOrReplaceTempView("censo_personas")

print("Ejecutando consulta SQL: Conteo de personas por Municipio:")

consulta_sql = """
SELECT
    Municipio,
    COUNT(*) AS Total_Personas
FROM
    censo_personas
GROUP BY
    Municipio
ORDER BY
    Total_Personas DESC
"""

resultado_sql = spark.sql(consulta_sql).show()


### **Regresión**

¿Existe relación entre la edad de la persona (PERNA01) y su nivel educativo alcanzado (NIVELEDU)?

PERNA01: edad (numérica continua).

NIVELEDU: nivel educativo (ordinal, pero codificado numéricamente).



![Modelos](https://th.bing.com/th/id/R.b6b5a3a73f568bd1f2b588feb93cd02a?rik=d0gt%2bmbw7VaQOw&riu=http%3a%2f%2fwww.favouriteblog.com%2fwp-content%2fuploads%2f2017%2f07%2fTypes-of-Learning.png&ehk=z%2btK2BpOxRNtB3u4Gis9bn3dIkKxuVwcqmENF9cGlx0%3d&risl=&pid=ImgRaw&r=0)


En esta práctica, continuaremos explorando los modelos supervisados, aquellos en los que el modelo aprende a partir de datos etiquetados. Estos modelos se dividen en:

- Regresión Lineal: Se usa cuando la variable objetivo es continua. Ejemplo: Predecir el precio de una vivienda en función de su superficie y ubicación.

-  Regresión Logística: Aunque su nombre sugiere "regresión", este modelo se usa para clasificación binaria o multiclase.
    - Calcula la probabilidad de que una observación pertenezca a una clase determinada.
    - Se usa en problemas como detección de spam, diagnóstico médico y predicción de abandono de clientes.

- Support Vector Machine (SVM): Es un modelo de clasificación que encuentra el hiperplano óptimo que separa las clases.
    - Funciona bien en problemas con datos complejos y espacios de alta dimensión.
    - Se usa en problemas como reconocimiento de imágenes y detección de fraudes.

Vamos a investigar las caracteristicas de la variable **NIVELEDU**


La siguiente tabla detalla la codificación para el máximo nivel educativo alcanzado (NIVELEDU) según la fuente de datos.

| Código (NIVELEDU) | Descripción (Máximo Nivel Educativo Alcanzado) |
| :---------------: | :--------------------------------------------- |
| 0                 | Menor de 4 años                                |
| 1                 | Preescolar                                     |
| 2                 | Primaria común o especial                      |
| 4                 | Educación media básica o Ciclo Básico          |
| 5                 | Educación media superior o Bachillerato        |
| 6                 | Capacitaciones o cursos de UTU                 |
| 7                 | Magisterio o profesorado                       |
| 8                 | Terciario no universitario                     |
| 9                 | Universidad o similar                          |
| 10                | Postgrado (Diploma/Maestría/Doctorado)         |
| 12                | Nunca asistió                                  |

Ver el tipo con schema detallado

In [ ]:
sdf_reg.printSchema()

Revisar estadísticos descriptivos

In [ ]:
sdf_reg.describe().show()

Contar cuántos valores distintos existen en la columna

In [ ]:
sdf_reg.select(countDistinct('NIVELEDU')).show()

Contar frecuencia de cada valor

In [ ]:
sdf_reg.groupBy('NIVELEDU').count().orderBy('NIVELEDU').show()


Excluir los valores
|    8888|
|    9898|

In [ ]:
sdf_reg = sdf_reg.filter(~col("NIVELEDU").isin([12, 8888, 9898]))


Verificar

In [ ]:
sdf_reg.select('NIVELEDU').distinct().orderBy('NIVELEDU').show()


Vamos a investigar las caracteristicas de la variable **PERNA01**


Variable que tiene información de la edad en años.

Descripción:
- Edad en años cumplidos. Las personas residentes en localidades con menos de 10.000 habitantes no tienen dato en esta variable por secreto estadístico.

- Población estimada (excluyendo los residentes en localidades de menos de 10.000 habitantes)


Contar cuántos valores distintos existen en la columna

In [ ]:
sdf_reg.select(countDistinct('PERNA01')).show()

Ver los valores únicos

In [ ]:
sdf_reg.select('PERNA01').distinct().orderBy('PERNA01').show()


Contar frecuencia de cada valor

In [ ]:
sdf_reg.groupBy('PERNA01').count().orderBy('PERNA01').show()

Primeros 30 valores ordenados ascendentemente

In [ ]:
sdf_reg.groupBy('PERNA01').count().orderBy('PERNA01').show(30)

Últimos 30 valores (mayores)

In [ ]:
sdf_reg.groupBy('PERNA01').count().orderBy(desc('PERNA01')).show(30)

Excluir los valores menores a 109


In [ ]:
sdf_reg = sdf_reg.filter(col("PERNA01") < 109)


Verificamos

In [ ]:
sdf_reg.groupBy('PERNA01').count().orderBy(desc('PERNA01')).show(30)

In [ ]:
sdf_reg.printSchema()

Ver coeficientes de correlación

In [ ]:
from scipy.stats import pearsonr

sdf_reg.select(corr("NIVELEDU", "PERNA01")).show()

In [ ]:
pdf = sdf_reg.select("NIVELEDU", "PERNA01").toPandas()


r, p = pearsonr(pdf["NIVELEDU"], pdf["PERNA01"])
print(f"r = {r:.4f}, p = {p:.6f}")


**Interpretación del coeficiente de correlación de Pearson**

El análisis de correlación de **Pearson** permite cuantificar la **fuerza y dirección** de la relación lineal entre dos variables numéricas.  
En este caso, estamos evaluando la relación entre:

- **NIVELEDU** → nivel educativo (variable dependiente)  
- **PERNA01** → indicador predictor (por ejemplo, una variable socioeconómica o demográfica del censo)

----

**r =** Es el coeficiente de correlación de Pearson. Toma valores entre -1 y +1.

**p =** Es el valor de significancia asociado al test estadístico. Permite evaluar si la correlación observada podría deberse al azar.


Cómo interpretar los resultados:
- Si r > 0, la relación es positiva: cuando aumenta PERNA01, también tiende a aumentar NIVELEDU.
- Si r < 0, la relación es negativa: cuando aumenta PERNA01, NIVELEDU tiende a disminuir.
- Cuanto más cerca de ±1, más fuerte es la relación lineal.
- Un valor cercano a 0 indica ausencia de relación lineal (aunque puede haber relación no lineal).

---
¿Cómo interpretar estos resultados?

Existe una correlación positiva débil pero significativa entre PERNA01 y NIVELEDU.

Esto sugiere que, en promedio, a medida que aumenta PERNA01, el nivel educativo también tiende a aumentar ligeramente, aunque la relación no es fuerte.

### **AHORA SI: Regresión**

Objetivo

- Evaluar si la edad predice el nivel educativo.

En este ejemplo haremos:
- Variable dependiente (Y): NIVELEDU
- Variable independiente (X): PERNA01 (Edad)

### Preparar los Datos para la Regresión Lineal (Sin Estandarización)

En este paso, prepararemos los datos para la regresión lineal utilizando VectorAssembler.


In [ ]:
sdf_reg = sdf_reg.withColumn("PERNA01", col("PERNA01").cast("double"))
sdf_reg = sdf_reg.withColumn("NIVELEDU", col("NIVELEDU").cast("double"))

In [ ]:
sdf_reg = sdf_reg.na.drop(subset=["PERNA01", "NIVELEDU"])


In [ ]:
assembler = VectorAssembler(inputCols=["PERNA01"], outputCol="features")

In [ ]:
assembled_no_scale = assembler.transform(sdf_reg)


**Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)**


Para evaluar el rendimiento del modelo, se dividirá el dataset en dos subconjuntos:

🔹 Conjunto de entrenamiento (Train): Usado para entrenar el modelo.

🔹 Conjunto de prueba (Test): Usado para evaluar el rendimiento del modelo.

In [ ]:
train_A, test_A = assembled_no_scale.randomSplit([0.8, 0.2], seed=42)

In [ ]:
print(f"Training Data Count: {train_A.count()}")

Mostrar la cantidad de registros en cada conjunto

In [ ]:
print(f"Test Data Count: {test_A.count()}")

### Entrenar el Modelo de Regresión Lineal

Ahora, crearemos y entrenaremos el modelo de regresión lineal.

Crear el modelo de regresión lineal

In [ ]:
lr_A = LinearRegression(featuresCol="features", labelCol="NIVELEDU")


Entrenar el modelo con los datos de entrenamiento

In [ ]:
lr_model_A = lr_A.fit(train_A)

### Evaluar el Modelo

Utilizaremos el conjunto de prueba para evaluar el modelo calculando el Error Cuadrático Medio (RMSE) y el coeficiente de determinación (R2).

In [ ]:
pred_A = lr_model_A.transform(test_A)
evaluator = RegressionEvaluator(labelCol="NIVELEDU", predictionCol="prediction")

rmse_A = evaluator.setMetricName("rmse").evaluate(pred_A)
r2_A   = evaluator.setMetricName("r2").evaluate(pred_A)

**Métricas del modelo**

- **RMSE (Root Mean Squared Error):** mide el error promedio de las predicciones del modelo. Valores más bajos indican un mejor ajuste.  

- **R² (Coeficiente de determinación):** representa la proporción de la variabilidad de **NIVELEDU** que puede explicarse a partir de **PERNA01**. Un R² más cercano a 1 indica un mejor ajuste del modelo.

### Inspeccionar los Coeficientes del Modelo

Examinar los coeficientes e intercepto del modelo

In [ ]:
print(f"RMSE (test): {rmse_A:.6f}")
print(f"R²   (test): {r2_A:.6f}")
print(f"Intercepto: {lr_model_A.intercept:.6f}")
print("Coeficientes:", lr_model_A.coefficients)

**Resultados del modelo de regresión lineal**

*Coeficientes del modelo*

| Variable | Coeficiente | Interpretación |
|-----------|-------------|----------------|
| PERNA01 | 0.019 | Por cada unidad que aumenta el valor de **PERNA01**, el nivel educativo promedio (**NIVELEDU**) aumenta en aproximadamente **0.019 puntos**, manteniendo constante el resto de variables. Esto indica una **relación positiva**: a mayor valor en PERNA01, ligeramente mayor nivel educativo. |



*Intercepto del modelo*

| Intercepto | Valor | Interpretación |
|-------------|--------|----------------|
| Intercepto | 3.839 | Cuando **PERNA01 = 0**, el valor estimado del nivel educativo (**NIVELEDU**) es **3.839**. Este representa el valor base del modelo, es decir, el punto donde la recta de regresión corta el eje Y. |

Interpretación general

El modelo muestra una **relación lineal positiva y débil** entre **PERNA01** y **NIVELEDU**: a medida que **PERNA01** aumenta, el nivel educativo promedio también tiende a incrementarse ligeramente.  

El **R²** obtenido permite estimar cuánta variabilidad del nivel educativo se explica por **PERNA01**.

Si el valor de R² es bajo, significa que aunque existe una tendencia positiva, **la variable PERNA01 por sí sola no explica una gran proporción de las diferencias en el nivel educativo**.

Si el rendimiento del modelo no cumple con sus expectativas, puede probar las siguientes estrategias para mejorarlo:

- **Escalado de funciones:** estandarice o normalice las funciones de entrada para garantizar que estén en la misma escala.

- **Ajuste de hiperparámetros:** ajuste los hiperparámetros del modelo, como la intensidad de la regularización o el recuento de iteraciones.

---
---

---

---

## **Parte  II: MODELO B: Regresión con ROBUST SCALER**

Generar vector crudo para escalar


In [ ]:
assembler_raw = VectorAssembler(inputCols=["PERNA01"], outputCol="raw_features")
assembled_raw = assembler_raw.transform(sdf_reg)

Split reproducible antes de escalar

In [ ]:
train_raw, test_raw = assembled_raw.randomSplit([0.8, 0.2], seed=42)

In [ ]:
print(f"Training Data Count: {train_raw.count()}")
print(f"Test Data Count: {test_raw.count()}")

Ajustar RobustScaler SOLO con train y transformar ambos

In [ ]:
scaler = RobustScaler(inputCol="raw_features", outputCol="features")
scaler_model = scaler.fit(train_raw)

train_B = scaler_model.transform(train_raw)
test_B  = scaler_model.transform(test_raw)

Regresión lineal sobre features escaladas

In [ ]:
lr_B = LinearRegression(featuresCol="features", labelCol="NIVELEDU")
lr_model_B = lr_B.fit(train_B)

pred_B = lr_model_B.transform(test_B)

rmse_B = evaluator.setMetricName("rmse").evaluate(pred_B)
r2_B   = evaluator.setMetricName("r2").evaluate(pred_B)

Resultados MODELO B (RobustScaler)

In [ ]:
print(f"RMSE (test, escalado): {rmse_B:.6f}")
print(f"R²   (test, escalado): {r2_B:.6f}")
print(f"Intercepto: {lr_model_B.intercept:.6f}")
print("Coeficientes:", lr_model_B.coefficients)

Comparativa rápida

In [ ]:
print(f"A) Sin estandarizar  -> RMSE: {rmse_A:.6f} | R²: {r2_A:.6f}")
print(f"B) RobustScaler      -> RMSE: {rmse_B:.6f} | R²: {r2_B:.6f}")

# **Interpretación**



In [ ]:
spark.stop()

Referencias para profundizar en este campo:


[Pyspark Tutorial: Getting Started with Pyspark](https://www.datacamp.com/tutorial/pyspark-tutorial-getting-started-with-pyspark)

[Linear Regression in PySpark](https://medium.com/@roshmitadey/a-comprehensive-guide-to-linear-regression-in-pyspark-810fdaf5c17c)

[Regresión lineal de PySpark: cómo crear y evaluar modelos de regresión lineal utilizando PySpark MLlib](https://www.machinelearningplus.com/pyspark/pyspark-linear-regression/?utm_content=cmp-true)








